## Assignment 05

In [1]:
# notebooks/berlin_rent_prediction.ipynb

## 1. Daten laden und Überblick

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Laden des Datensatzes
# Stellen Sie sicher, dass 'immo_data.csv' im Ordner 'data/' liegt
try:
    df = pd.read_csv('../data/immo_data.csv')
    print("Datensatz erfolgreich geladen.")
except FileNotFoundError:
    print("Fehler: 'immo_data.csv' nicht gefunden. Stellen Sie sicher, dass die Datei im Ordner 'data/' liegt.")
    # Optional: Pfad anpassen oder Anweisungen zum Herunterladen geben

# Erster Überblick über die Daten
print("\n--- Daten-Info ---")
df.info()

print("\n--- Erste 5 Zeilen ---")
print(df.head())

print("\n--- Statistische Zusammenfassung ---")
print(df.describe())

## 2. Daten filtern: Nur Berlin

# Die Spalte 'regio2' oder 'regio3' könnte Berlin enthalten.
# Je nach Datensatz kann der Name der Spalte variieren.
# Es ist wichtig, die genaue Spalte zu identifizieren, die Städte/Regionen angibt.
# Oft ist 'regio2' für Bundesländer und 'regio3' für Städte/Landkreise.

# Beispiel basierend auf typischen Kaggle-Immobiliendaten:
# Angenommen, 'regio3' enthält Städtenamen und 'regio2' enthält Bundesländer
# Sie müssen dies anhand der tatsächlichen Spaltennamen in Ihrer 'immo_data.csv' überprüfen!

berlin_df = df[
    (df['regio2'] == 'Berlin') | # Manchmal ist Berlin direkt in regio2
    (df['regio3'] == 'Berlin')   # Manchmal ist Berlin in regio3
    ]

# Überprüfung, ob Daten für Berlin gefunden wurden
if berlin_df.empty:
    print("\nWarnung: Keine Daten für Berlin gefunden. Überprüfen Sie die Spaltennamen für die Region (z.B. 'regio2', 'regio3', 'city', 'state') und die Schreibweise von 'Berlin'.")
    # Versuchen Sie alternativ, die einzigartigen Werte der Regionalspalten anzuzeigen:
    # print(df['regio2'].unique())
    # print(df['regio3'].unique())
else:
    print(f"\n--- Daten für Berlin gefiltert ({len(berlin_df)} Einträge) ---")
    print(berlin_df.head())

## 3. Explorative Datenanalyse (EDA) - Berlin-spezifisch

# Hier würden Sie eine detailliertere Analyse der Berliner Daten durchführen.
# Beispiel: Verteilung der Mietpreise
plt.figure(figsize=(10, 6))
sns.histplot(berlin_df['price'], bins=50, kde=True)
plt.title('Verteilung der Kaltmieten in Berlin')
plt.xlabel('Kaltmiete (€)')
plt.ylabel('Anzahl der Angebote')
plt.show()

# Beispiel: Mietpreis pro Quadratmeter vs. Wohnfläche
plt.figure(figsize=(12, 7))
sns.scatterplot(x='livingSpace', y='price', data=berlin_df, alpha=0.6)
plt.title('Mietpreis vs. Wohnfläche in Berlin')
plt.xlabel('Wohnfläche (m²)')
plt.ylabel('Kaltmiete (€)')
plt.show()

# Umgang mit fehlenden Werten (Beispiel)
print("\n--- Fehlende Werte (Top 10) ---")
print(berlin_df.isnull().sum().sort_values(ascending=False).head(10))

## 4. Feature Engineering & Preprocessing (Vorbereitung für das Modell)

# Beispiel: Einfache Feature-Auswahl und Behandlung fehlender Werte
# Dies ist stark vereinfacht und muss an Ihren Datensatz angepasst werden!

# Auswahl relevanter Spalten (Beispielhafte Auswahl)
features = ['livingSpace', 'noRooms', 'yearConstructed', 'balcony', 'hasKitchen', 'garden', 'totalRent']
target = 'price' # Angenommen 'price' ist Ihre Kaltmiete

# Filtern der Berliner Daten auf relevante Features und den Target
berlin_model_df = berlin_df[features + [target]].copy()

# Umgang mit fehlenden Werten (Beispiel: Einfacher Imputations-Ansatz)
# Ersetzen Sie fehlende numerische Werte durch den Median
for col in ['livingSpace', 'noRooms', 'yearConstructed', 'totalRent']:
    if col in berlin_model_df.columns:
        berlin_model_df[col].fillna(berlin_model_df[col].median(), inplace=True)

# Ersetzen Sie fehlende Boolesche Werte (falls vorhanden) mit False oder 0
for col in ['balcony', 'hasKitchen', 'garden']:
    if col in berlin_model_df.columns:
        berlin_model_df[col].fillna(False, inplace=True) # Oder 0, je nach Datentyp

# Umwandlung von booleschen Werten in 0/1 (falls nicht schon geschehen)
for col in ['balcony', 'hasKitchen', 'garden']:
    if col in berlin_model_df.columns and berlin_model_df[col].dtype == 'bool':
        berlin_model_df[col] = berlin_model_df[col].astype(int)

# Entfernen von Zeilen mit NaN im Target (oder wenn nach Imputation noch NaN vorhanden sind)
berlin_model_df.dropna(subset=[target], inplace=True)

print(f"\n--- Daten nach Preprocessing ({len(berlin_model_df)} Einträge) ---")
print(berlin_model_df.info())

## 5. Modelltraining und Evaluierung (Skizze)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Aufteilen in Features (X) und Target (y)
X = berlin_model_df[features] # Target-Spalte sollte hier nicht enthalten sein
y = berlin_model_df[target]

# Sicherstellen, dass 'totalRent' nicht als Feature verwendet wird, wenn 'price' das Target ist
if 'totalRent' in X.columns and target == 'price':
    X = X.drop('totalRent', axis=1) # Beispielhafte Korrektur falls 'totalRent' ein Feature und 'price' Target ist

# Trainings- und Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

# Modell initialisieren und trainieren (Beispiel: Lineare Regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred = model.predict(X_test)

# Modell evaluieren
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- Modell-Evaluierung (Lineare Regression) ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

# Visualisierung der Vorhersagen vs. tatsächliche Werte
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2) # Idealvorhersage-Linie
plt.title('Tatsächliche vs. Vorhergesagte Kaltmieten')
plt.xlabel('Tatsächliche Kaltmiete (€)')
plt.ylabel('Vorhergesagte Kaltmiete (€)')
plt.grid(True)
plt.show()

# Nächste Schritte:
# - Erweiterte Feature Engineering Techniken (z.B. PLZ in Kategorien umwandeln, Interaktionen)
# - Behandlung von Ausreißern
# - Einsatz komplexerer Modelle (Random Forest, Gradient Boosting, Neuronale Netze)
# - Hyperparameter-Optimierung
# - Kreuzvalidierung
# - Detailliertere Fehleranalyse

Fehler: 'immo_data.csv' nicht gefunden. Stellen Sie sicher, dass die Datei im Ordner 'data/' liegt.

--- Daten-Info ---


NameError: name 'df' is not defined

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Laden des Datensatzes
# Stellen Sie sicher, dass 'immo_data.csv' im Ordner 'data/' liegt
try:
    df = pd.read_csv('../data/immo_data.csv')
    print("Datensatz erfolgreich geladen.")
except FileNotFoundError:
    print("Fehler: 'immo_data.csv' nicht gefunden. Stellen Sie sicher, dass die Datei im Ordner 'data/' liegt.")
    # Optional: Pfad anpassen oder Anweisungen zum Herunterladen geben

# Erster Überblick über die Daten
print("\n--- Daten-Info ---")
df.info()

print("\n--- Erste 5 Zeilen ---")
print(df.head())

print("\n--- Statistische Zusammenfassung ---")
print(df.describe())

Fehler: 'immo_data.csv' nicht gefunden. Stellen Sie sicher, dass die Datei im Ordner 'data/' liegt.

--- Daten-Info ---


NameError: name 'df' is not defined